## Homework 4 for ECON-GA 3002, Section 12.

**{Name, N-Number, Email}**: {Arnav Sood, N11193569, asood@nyu.edu}

**Overview**: Show the downward bias in the standard OLS estimate of the lag coefficient by running a large number of trials for a few (alpha, n) pairs, and then plotting. Optimize for speed.

**Disclosure**: Was not written using vim (sorry). Was instead written using a Jupyter notebook and REPL.

In [7]:
# Vanilla Python version. Before optimizing, get a vanilla code set in Python.

# Update: parallelization is hard.

# The optimization path will probably look something like this:
'''
    - Write vanilla Python code.
            - Wall Time: 2m41s
    - Make easy Python improvements (e.g, @jit, numbafy)
            - @jit: Wall Time: 17.9s
            - @jit on Mac (no VM): 11.3s
            - add numpy: 2s?
            - remove print: like 5s?
    - Make harder Python improvements (not sure what they are a priori)
    - Write code in C if it makes sense
    - Figure out what PEP8 is, and make sure I'm using it
'''

# Throw in numba jit compile.
from numba import jit

# Add more numpy stuff.
import numpy as np

# Setup variables.
alphas = np.arange(0.5,1,0.1)
ns = np.arange(50,550,50)

# Define a function to take an (alpha, n, beta) tuple, and return the bias for the estimate.

# Store this as a lambda function so it can compile the operation once. (?)
estimate = lambda sumx, sumy, sumxsq, sumxy, n : (sumxy - (n-1)*(sumx/(n-1))*(sumy/(n-1)))/(sumxsq - (n-1)*math.pow((sumx/(n-1)),2));

@jit
def bias(alpha, n, beta):
        sumx = 0
        sumy = 0
        sumxy = 0
        sumxsq = 0
        x = rand()*100
        for i in range(n):
            sumx += x
            sumxsq += math.pow(x,2)
            y = beta + alpha*x + rand()
            sumxy += x*y
            sumy += y
            x = y
        val = estimate(sumx, sumy, sumxsq, sumxy, n) - alpha
        return val

# Define a function to take an (alpha, n, beta) tuple, estimate 10,000 times, and then average to get the expected bias.
from numpy.random import normal as rand
import math

@jit
def calcbias(alpha, n, beta):
    biases = np.zeros(10000)
    for i in range(10000):
        biases[i] = bias(alpha,n,beta)
    avg = np.mean(biases)
    return avg

# Define a function to call the above on the right sets of params, and return the biases.
@jit
def main():
    results = np.zeros(50)
    for i in range(5):
        for j in range(10):
            results[10*i+j] = calcbias(alphas[i],ns[j],1)
    print(results)
    return results


In [8]:
%time x = main()

[-0.00773579 -0.00451932 -0.00405684 -0.0032614  -0.00247437 -0.00290046
 -0.00247151 -0.00230902 -0.00192862 -0.00184639 -0.00955591 -0.00605299
 -0.00431926 -0.00329249 -0.00320002 -0.00301772 -0.00254632 -0.00254722
 -0.00242114 -0.00199767 -0.01012679 -0.00598315 -0.00470899 -0.00380652
 -0.00334048 -0.00328217 -0.00292817 -0.00276288 -0.00245581 -0.00245079
 -0.0134482  -0.00737448 -0.0051959  -0.00462799 -0.00407733 -0.00350783
 -0.00310341 -0.00270505 -0.00256782 -0.00253409 -0.03771731 -0.01072232
 -0.00666914 -0.00580565 -0.00465095 -0.00411568 -0.00345366 -0.00309649
 -0.00295633 -0.00281806]
CPU times: user 11.3 s, sys: 24.8 ms, total: 11.3 s
Wall time: 11.3 s


In [ ]:
# Plot.
from matplotlib import pyplot as plt
vec1 = x[0:10]
vec2 = x[10:20]
vec3 = x[20:30]
vec4 = x[30:40]
vec5 = x[40:50]

# Plot each of the vectors against 'n'
plt.plot(ns, vec1, 'r-', ns, vec2, 'b-', ns, vec3, 'o-',ns,vec4,'g-',ns,vec5,'k-')
plt.title('Biases vs. Sample Sizes in OLS Estimate of Lag Coefficient')
plt.xlabel('Sample Size')
plt.ylabel('Bias (Absolute Difference)')
plt.show()

![title](figure_1.png)

## What We See

There are a few properties of the OLS estimator which we can extract from the figure:

**Consistency**: There is a clear asymptotic trend to 0 bias. This is because the OLS estimator is a consistent one, which means that it has exactly this property.

**Downward 